In [81]:
#Problems Encountered
'''
1. Abbreviation - B.A. B.S. - after my punctuation remover, it becomes B A B S. Making it miss stuff
2. Tokenization - Miss key words with two or more words like "React Native"
3. Double spaces - not completely sure yet but I think it's from the punctuation remover, but 
the regex that removes multiple consecutive spaces can deal with that
'''


'\n1. Abbreviation - B.A. B.S. - after my punctuation remover, it becomes B A B S. Making it miss stuff\n2. Tokenization - Miss key words with two or more words like "React Native"\n3. Double spaces - not completely sure yet but I think it\'s from the punctuation remover, but \nthe regex that removes multiple consecutive spaces can deal with that\n'

In [1]:
import re
from nltk.tokenize import word_tokenize 

In [22]:
#Seperate Job Posts, put them all in a list, with each element a job post
#Preprocessing like removing punctuations, lowercase everything
def preprocessing(jobs_file):
    jobsFile = open(jobs_file)
    jobslines = jobsFile.readlines()
    origPosts = []
    jobPosts = []
    id_list = []
    post = ""
    origPost = ""
    firstOne = True
    for l in range(len(jobslines)):
        if "BREAK" in jobslines[l]:
            id_list.append(int(jobslines[l][5:]))
            if firstOne:
                firstOne = False
            else:
                jobPosts.append(post)
                origPosts.append(origPost)
                post = ""
                origPost = ""
            continue
        origPost += jobslines[l]
        if l == len(jobslines)-1:
            jobPosts.append(post)
            origPosts.append(origPost)
       # post += jobslines[l].replaceAll("[\\p{Punct}&&[^.]]", "").lower();
        post+= re.sub(r'[^\w\s\+]', ' ', jobslines[l].lower())

    print('----')
    for p in range(len(jobPosts)):
        jobPosts[p]= re.sub(' +', ' ', jobPosts[p].replace('\n',' ')) #remove unnecessary double/triple white space
    # for p in range(len(jobPosts)):
    #     jobPosts[p] = word_tokenize(jobPosts[p])
    return origPosts, jobPosts, id_list

In [23]:
def read_termsFile(termsFileName):
    termsFile = open(termsFileName,"r")
    terms = termsFile.readlines()
    print('opened:',termsFileName)
    print('number terms:',len(terms))
    keywords = [term.rstrip('\n').lower() for term in terms]
    return keywords

In [4]:
max_spaces = 0
max_term = ""
for term in terms:
    count = 0
    for tok in term:
        if tok == ' ':
            count+=1
    if count > max_spaces:
        max_spaces = count
        max_term = term
print('max spaces in a word in the termsFile',max_spaces)
print('the word:', max_term)

NameError: name 'terms' is not defined

In [24]:
def extract_seniority(o_P):
    seniority_levels = ['Internship','Entry level','Associate','Mid-Senior level','Director','Executive']
    s_tag = ""
    s_boo =False
    #Some weird inconsistency with linkedin seniority level upperlower caseness, edge case
    if "Seniority Level" in o_P:
        s_tag = "Seniority Level"
        s_boo = True
    elif "Seniority level" in o_P:
        s_tag = "Seniority level"
        s_boo = True
    if s_boo:
        seniorityIndex = o_P.index(s_tag) +1 
        if o_P[seniorityIndex] in seniority_levels:
            #print('SENIORITY:', seniority_levels[seniority_levels.index(o_P[seniorityIndex])])
            return seniority_levels[seniority_levels.index(o_P[seniorityIndex])]
        elif o_P[seniorityIndex] == "Not Applicable":
            return -1
        else:
            print("Doesn't exist in seniority levels, weird.")
            return -1
    else:
        return -1

def extract_degree_lvl(post):
    deg_lvls = []
    deg_lvl = {
               'a':['associate\'s','associate degree'],
               'b':['ba','bs','bachelor','bachelor\'s','b s'],
               'm':['master','ms','master\'s','m s'],
               'p':['ph d','phd']

    }
    for d_key, d_value in deg_lvl.items():
        for variation in d_value:
            variation = " "+variation+" "
            if variation in post:
                deg_lvls.append(d_key)
    #print('EDUCATION LEVEL',set(deg_lvls))
    return list(set(deg_lvls))
    
def extract_degree_title(post):
    deg_titles = []
    deg_title = {'computer science':['computer science','cs','c s'],
                 'computer engineering':['computer engineering','ce','c e'],
                  'electrical engineering':['electrical engineering','ee','e e'],
                 'applied math':['applied math','applied mathematics'],
                 'physics':['physics'],
                 'statistics':['statistics'],
                 'bioinformatics/comp-bio':['bioinformatics','computational biology']

    }

    for d_key, d_value in deg_title.items():
        for title in d_value:
            title = " "+title+" "
            if title in post:
                deg_titles.append(d_key)
       
   #print('DEGREE TITLE', set(deg_titles))
    return list(set(deg_titles))

def extract_tech_terms(post,keywords):
    skills = []
    for key in keywords:
        key= " "+key+" "
        if key in post:
            skills.append(key.strip())
   #print('SKILLS',set(skills))
    return list(set(skills))

def extract_languages(post,languages):
    lang = []
    for key in languages:
        key= " "+key+" "
        if key in post:
            lang.append(key.strip())
   #print('SKILLS',set(skills))
    return list(set(lang))


def extract_industry(o_P,linkedin_industries):

    s_tag = ""
    s_boo =False
    industries = []
    #Some weird inconsistency with inconsistency industry(ies) plural/singular, edge case
    if "Industry" in o_P:
        s_tag = "Industry"
        s_boo = True
    elif "Industries" in o_P:
        s_tag = "Industries"
        s_boo = True
    if s_boo:
        industryIndex = o_P.index(s_tag) +1 
        for ind in linkedin_industries:
            if ind in o_P[industryIndex] or ind.replace("&","and") in o_P[industryIndex]:
                industries.append(ind)
        #print('INDUSTRIES:', industries)
        return industries
def extract_yoe(post):
    tokenize = post.split(' ')
    yoe_variation = ['years of experience','years of full time','years full time','years industry experience','years of industry experience','years work experience','years of work experience']
    for w in range(len(tokenize)):
        if tokenize[w].isdigit():
            #print(tokenize[w-5:w],tokenize[w],tokenize[w+1:w+5])
            for yv in yoe_variation:
                if yv in ' '.join(tokenize[w:w+6]):
                    #print('YOE:', tokenize[w])
                    return int(tokenize[w])
    #print('YOE:','N/A')
    return -1

In [27]:
def data_extraction(origPosts, jobPosts,keywords,id_list,linkedin_industries,languages):    
    output = {}
    for i in range(len(jobPosts)):  
        output[id_list[i]] = {}
        o_P = origPosts[i].split('\n')
        #SENIORITY
        output[id_list[i]]['seniority']=extract_seniority(o_P)
        #INDUSTRY
        output[id_list[i]]['industry']=extract_industry(o_P,linkedin_industries)
        #Keyword Extraction -----
        #Languages
        output[id_list[i]]['languages'] = extract_languages(jobPosts[i],languages)
        #TECH SKILLS
        output[id_list[i]]['skills']=extract_tech_terms(jobPosts[i], keywords)
        #Degree level
        output[id_list[i]]['educationLevel']=extract_degree_lvl(jobPosts[i])
        #DEGREE TITLE
        output[id_list[i]]['degreeTitle']=extract_degree_title(jobPosts[i])
        #YoE
        output[id_list[i]]['yoe']=extract_yoe(jobPosts[i])
        print()
        dbup_table = output[id_list[i]].copy()
        dbup_table['jobID'] = id_list[i]
       # dbup_table['table'] = "Web Developer"
       # db_uploadFunction(dbup_table)
    return output
    


In [28]:
template = {'4385545':{'skills':['javascript,python, angular'], 
                       'industry':['healthcare','telecommunication'],
                       'education level':['bachelor','master'],
                       'seniority':'Senior',
                       'Years of Experience':5,
                       'Degree Title':['ce','cs']
                      },
            '5345465':{
                
            }
           }

In [29]:
orig_posts, jobPosts, id_list = preprocessing("sample_jobposts.txt")
keywords = read_termsFile("lists/final_keywords2.txt")
languages = read_termsFile("lists/topLanguages.txt")
linkedin_industries = open("lists/linkedin_industries.txt",'r').read().split('\n')
print('opened lists/linkedin_industries.txt:\nnumber of terms',len(linkedin_industries))
output = data_extraction(orig_posts, jobPosts, keywords,id_list,linkedin_industries,languages)
import pprint

pprint.pprint(output)
#print(output)

----
opened: lists/final_keywords2.txt
number terms: 2094
opened: lists/topLanguages.txt
number terms: 43
opened lists/linkedin_industries.txt:
number of terms 147







{12345: {'degree_level': ['b'],
         'degree_title': ['computer science'],
         'industries': ['Information Technology and Services',
                        'Computer Software',
                        'Internet'],
         'languages': ['java', 'javascript'],
         'seniority': 'N/A',
         'skills': ['communication',
                    'product',
                    'database',
                    'web application',
                    'zoom',
                    'less',
                    'software development',
                    'linux',
                    'testing',
                    'mvc'],
         'yoe': -1},
 23435: {'degree_level': ['b'],
         'degree_title': ['computer science'],
         'industries': None,
         'languages': ['ruby', 'typescript', 'postgresql', 'mysql', 'nosql

In [104]:
tokenize = jobPosts[0].split(' ')
for w in range(len(tokenize)):
    if tokenize[w].isdigit():
        print(tokenize[w-5:w],tokenize[w],tokenize[w+1:w+5])

['been', 'recognized', 'by', 'comparably', 'as'] 1 ['ceo', 'company', 'happiness', 'benefits']
['large', 'company', 'us', 'ceo', 'in'] 2018 ['wealthfront', 'and', 'business', 'insider']
['internship', 'or', 'related', 'work', 'experience'] 3 ['years', 'work', 'experience', 'have']
['been', 'recognized', 'by', 'comparably', 'as'] 1 ['ceo', 'company', 'happiness', 'benefits']
['large', 'company', 'us', 'ceo', 'in'] 2018 ['wealthfront', 'and', 'business', 'insider']


In [89]:
scraped_in = scraped_in.split('\t')
for s in range(len(scraped_in)):
    scraped_in[s] = scraped_in[s].replace("\n","")
    scraped_in[s] = ''.join([i for i in scraped_in[s] if not i.isdigit()])
scraped_in[0] = "Information Technology and Services"

AttributeError: 'list' object has no attribute 'split'

In [39]:
scraped_in = scraped_in[:-1]

In [60]:
"Marketing & Advertising".replace("&","and")

'Marketing and Advertising'

In [80]:
' '.join(['years','of'])

'years of'

In [21]:
import re
s = "string. With.++ Punctuation? C/C++ Java, Go, C, C++"
print(s)
s = re.sub(r'[^\w\s\+]',' ',s)
print(s)
s = re.sub(' +', ' ',s)
print(s)

string. With.++ Punctuation? C/C++ Java, Go, C, C++
string  With ++ Punctuation  C C++ Java  Go  C  C++
string With ++ Punctuation C C++ Java Go C C++
